In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
plot_params = dict(surface='white', hemi='both', verbose=0)


Get Forward model

In [2]:
info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.2s remaining:    2.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.2s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


In [3]:
n_samples = 10000
settings = dict(n_sources=(1, 10), extents=(15, 25), duration_of_trial=0.1)
sim = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [6]:
net = Net(fwd, verbose=1).fit(sim)

(10, 61) 1284
Model: "net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 500)           1124000   
_________________________________________________________________
flatten (Flatten)            (None, 5000)              0         
_________________________________________________________________
dense (Dense)                (None, 12840)             64212840  
_________________________________________________________________
reshape (Reshape)            (None, 10, 1284)          0         
Total params: 65,336,840
Trainable params: 65,336,840
Non-trainable params: 0
_________________________________________________________________
(10000, 10, 61) (10000, 10, 1284)
Epoch 1/100
71/71 [==============================] - 37s 505ms/step - loss: 0.0602 - mean_squared_error: 1.0033 - val_loss: 0.0599 - val_mean_squared_error: 1.0016
Epoch 2/100
71/71 [================

### Evaluate

In [ ]:
sim_test = Simulation(fwd, info, settings=settings).simulate(1)
idx = 0
prediction = net.predict(sim_test)
sim_test.source_data[0].plot(**plot_params)
prediction.plot(**plot_params)




In [ ]:
tf.keras.backend.set_image_data_format('channels_last')
lstm = layers.LSTM(5, return_sequences=True, input_shape=(n_timepoints, n_channels))
out = 
lstm(x).shape

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
x = sim.eeg_data.get_data()
y = np.stack([source.data for source in sim.source_data], axis=0)
x = np.swapaxes(x,1,2)
y = np.swapaxes(y,1,2)

print(x.shape, y.shape)
n_samples, n_timepoints, n_channels = x.shape
n_dipoles = y.shape[2]
n_units = 5
input_shape = (n_timepoints, n_channels)
print(input_shape)

model = tf.keras.models.Sequential()
model.add(layers.InputLayer(input_shape=input_shape))
model.add(layers.LSTM(n_units, return_sequences=True, input_shape=(n_timepoints, n_channels)))
model.add(layers.Flatten())
model.add(layers.Dense(int(n_timepoints*n_dipoles), activation='relu'))
model.add(layers.Reshape((n_timepoints, n_dipoles)))

model.build(input_shape=input_shape)
model.compile('adam', loss='mean_squared_error')
model.fit(x,y, batch_size=64)
model.summary()


In [ ]:
# Test if old net still works
settings_old = dict(n_sources=(1, 10), extents=(15, 25))
sim_old = Simulation(fwd, info, settings=settings_old).simulate(n_samples=5000)
net = Net(fwd).fit(sim_old)
net.predict(sim_old).plot(**plot_params)
sim_old.source_data.plot(**plot_params)
